# Example of BOMD MD (NVE, NVT)

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin

torch.set_default_dtype(torch.float64)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                                           # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                           # [1], adaptive mixing
                                           # [1, K, L, M] # advanced adaptive mixing.
                                           # First, it does linear mixing for M steps. Mixing coeff is K the first 5 SCF steps.
                                           # Then it incrementally goes to L and becomes L at M-5 SCF step. From M-5 to M it's equal to L. After M'th SCF step, adaptive mixing begins.
                                           # 
                                           # [2], adaptive mixing, then pulay
                   }

output = {
'molid': [0,1],
'prefix': f'Outputs/3_MD_BOMD_Basic',
'print_every': 1,
# write a .xyz file
"xyz": {
    "every": 1    # write .xyz file every N steps; 0 disables
    },
# write an HDF5 file
"h5": {
    "data_every": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "dynamics_vectors_every": 1,   # write vel/forces/coords; 0 disables
    "write_mo": False,
    },
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   383.21   9.906683e-02 -4.755775e+02 -4.754784e+02 ||    413.24   1.068305e-01 -1.091397e+02 -1.090329e+02 || 
     2   356.35   9.212293e-02 -4.755705e+02 -4.754784e+02 ||    489.10   1.264418e-01 -1.091595e+02 -1.090330e+02 || 
     3   322.15   8.328269e-02 -4.755616e+02 -4.754783e+02 ||    617.13   1.595401e-01 -1.091929e+02 -1.090333e+02 || 
     4   283.80   7.336851e-02 -4.755516e+02 -4.754783e+02 ||    779.04   2.013979e-01 -1.092351e+02 -1.090337e+02 || 
     5   244.53   6.321664e-02 -4.755414e+02 -4.754782e+02 ||    950.32   2.456757e-01 -1.092799e+02 -1.090342e+02 || 
     6   207.27   5.358287e-02 -4.755318e+02 -4.754782e+02 ||   1102.78   2.850895e-01 -1.093198e+02 -1.090347e+02 || 
     7   174.33   4.506810e-02 -4.755232e+02 -4.754782e+02 ||   1208.25   3.123568e-01 -1.093475e+02 -1.090351e+02 || 
     8   147.27   3.807169e-02 -4.755162e+02 -4.754782e+02 ||   1243.20   3.213914e-01 -1.093568e+02 -1.090354e+

## *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created
## *MD_BOMD_BASIC.0.h5* and *MD_BOMD_BASIC.1.h5* are HDF5 files that contain all the data of the molecular dynamics trajectory

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [3]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                   }

output = {
'molid': [0,1],
'prefix': 'Outputs/3_MD_BOMD_Basic_E_cntrl',
'print_every': 1,
# write a .xyz file
"xyz": {
    "every": 1    # write .xyz file every N steps; 0 disables
    },
# write an HDF5 file
"h5": {
    "data_every": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "dynamics_vectors_every": 1,   # write vel/forces/coords; 0 disables
    "write_mo": False,
    },
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, control_energy_shift=True, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   261.39   6.757371e-02 -4.742765e+02 -4.742089e+02 ||    549.92   1.421656e-01 -1.084612e+02 -1.083190e+02 || 
     2   229.34   5.929003e-02 -4.742682e+02 -4.742089e+02 ||    852.18   2.203052e-01 -1.085393e+02 -1.083190e+02 || 
     3   310.21   8.019576e-02 -4.742891e+02 -4.742089e+02 ||   1287.93   3.329566e-01 -1.086520e+02 -1.083190e+02 || 
     4   508.24   1.313903e-01 -4.743403e+02 -4.742089e+02 ||   1824.87   4.717650e-01 -1.087908e+02 -1.083190e+02 || 
     5   824.45   2.131368e-01 -4.744220e+02 -4.742089e+02 ||   2415.65   6.244933e-01 -1.089435e+02 -1.083190e+02 || 
     6  1254.49   3.243107e-01 -4.745332e+02 -4.742089e+02 ||   2997.03   7.747923e-01 -1.090938e+02 -1.083190e+02 || 
     7  1786.35   4.618068e-01 -4.746707e+02 -4.742089e+02 ||   3490.31   9.023143e-01 -1.092213e+02 -1.083190e+02 || 
     8  2398.27   6.200006e-01 -4.748289e+02 -4.742089e+02 ||   3805.37   9.837652e-01 -1.093028e+02 -1.083190e+

## Scale velocities to control temperature

In [4]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                   }

output = {
'molid': [0,1],
'prefix': 'Outputs/3_MD_BOMD_Basic_T_cntrl',
'print_every': 1,
# write a .xyz file
"xyz": {
    "every": 1    # write .xyz file every N steps; 0 disables
    },
# write an HDF5 file
"h5": {
    "data_every": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "dynamics_vectors_every": 1,   # write vel/forces/coords; 0 disables
    "write_mo": False,
    },
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   1.034080e-01 -4.743593e+02 -4.742559e+02 ||    400.00   1.034080e-01 -1.083705e+02 -1.082671e+02 || 
     2   400.00   1.034080e-01 -4.744226e+02 -4.743192e+02 ||    400.00   1.034080e-01 -1.083512e+02 -1.082478e+02 || 
     3   400.00   1.034080e-01 -4.744967e+02 -4.743933e+02 ||    400.00   1.034080e-01 -1.083783e+02 -1.082749e+02 || 
     4   400.00   1.034080e-01 -4.745769e+02 -4.744735e+02 ||    400.00   1.034080e-01 -1.084460e+02 -1.083426e+02 || 
     5   400.00   1.034080e-01 -4.746599e+02 -4.745565e+02 ||    400.00   1.034080e-01 -1.085378e+02 -1.084344e+02 || 
     6   400.00   1.034080e-01 -4.747430e+02 -4.746396e+02 ||    400.00   1.034080e-01 -1.086398e+02 -1.085364e+02 || 
     7   400.00   1.034080e-01 -4.748246e+02 -4.747212e+02 ||    400.00   1.034080e-01 -1.087434e+02 -1.086400e+02 || 
     8   400.00   1.034080e-01 -4.749034e+02 -4.748000e+02 ||    400.00   1.034080e-01 -1.088438e+02 -1.087403e+

## Langevin Thermostat

In [5]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                   }

output = {
'molid': [0,1],
'prefix': 'Outputs/3_MD_BOMD_Langevin',
'print_every': 1,
# write a .xyz file
"xyz": {
    "every": 1    # write .xyz file every N steps; 0 disables
    },
# write an HDF5 file
"h5": {
    "data_every": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "dynamics_vectors_every": 1,   # write vel/forces/coords; 0 disables
    "write_mo": False,
    },
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# specify damping factor
md =  Molecular_Dynamics_Langevin(damp = 100.0, seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   409.37   2.116600e-01 -4.755801e+02 -4.753684e+02 ||    460.43   2.380608e-01 -1.091545e+02 -1.089165e+02 || 
     2   378.71   1.958077e-01 -4.755594e+02 -4.753635e+02 ||    506.31   2.617822e-01 -1.091830e+02 -1.089212e+02 || 
     3   290.06   1.499745e-01 -4.755248e+02 -4.753748e+02 ||    551.07   2.849252e-01 -1.092147e+02 -1.089297e+02 || 
     4   212.63   1.099367e-01 -4.754848e+02 -4.753748e+02 ||    638.74   3.302523e-01 -1.092460e+02 -1.089158e+02 || 
     5   153.65   7.944377e-02 -4.754485e+02 -4.753690e+02 ||    732.13   3.785385e-01 -1.092689e+02 -1.088903e+02 || 
     6   103.13   5.332426e-02 -4.754234e+02 -4.753701e+02 ||    711.03   3.676292e-01 -1.092744e+02 -1.089068e+02 || 
     7    82.28   4.254034e-02 -4.754154e+02 -4.753729e+02 ||    700.39   3.621302e-01 -1.092640e+02 -1.089018e+02 || 
     8   104.31   5.393153e-02 -4.754233e+02 -4.753693e+02 ||    670.04   3.464357e-01 -1.092415e+02 -1.088951e+